In [1]:
import tenzing
from tenzing.core.models import model_relation
from tenzing.core.typesets import tenzing_typeset, tenzingTypeset
from tenzing.core.model_implementations import *
from collections import Counter
import pandas as pd
import numpy as np
from shapely import wkt
import datetime
from IPython.core.display import display, HTML
%load_ext autoreload
%autoreload 2

In [2]:
x = tenzingTypeset([tenzing_float, tenzing_integer])
x.relation_map

{tenzing_float: {tenzing_integer: <tenzing.core.models.model_relation at 0x11b8dada0>},
 tenzing_integer: {}}

In [3]:
from tenzing.core.model_implementations.typesets import tenzing_standard

df = pd.DataFrame({'Col 1': range(3), 
                   'Col 2': [2 * i for i in range(3)], 
                   'Col 3': ['howdy', 'howdy', 'doody'],
                   'times': pd.to_datetime([datetime.date(2011,1,1), datetime.date(2012, 1, 1), datetime.date(2013,1,1)])})
x = tenzing_standard()
x.prep(df)
summer = x.summarize(df)


You forgot to implement handling for multiple matches. Go fix that retard


In [4]:
x = tenzing_standard()
x.prep(df)
y = x.summary_report(df)
y

You forgot to implement handling for multiple matches. Go fix that retard


Number of Observations,3
Number of Variables,4
tenzing_integer,2
tenzing_string,1
tenzing_timestamp,1
nunique,3.0
mean,1.0
std,1.0
max,2.0
min,0.0
median,1.0


In [5]:
p = HTML(y.generate_html())
p.

SyntaxError: invalid syntax (<ipython-input-5-4ce9a183720e>, line 2)

In [ ]:

from pandas_profiling import ProfileReport
ProfileReport(df)


In [ ]:
from tenzing.core.models import tenzing_model

type(tenzing_model)

In [ ]:
tenzing_integer

In [ ]:
import pandas as pd
import numpy as np
x = pd.Series([1,2,3, np.nan])

print(x in tenzing_integer)
print(x in tenzing_float)

In [ ]:
int_float_relation = model_relation(tenzing_float, lambda x: False, lambda x: False)
tenzing_float.register_relation(int_float_relation)

In [ ]:
import numpy as np

x = pd.Series(list(range(10000)))
%timeit x.astype(np.float)
%timeit x.astype(float)

In [ ]:
x.astype(int).nbytes

In [ ]:
test_series = pd.Series([pd.datetime(2010, 1, 1), pd.datetime(2010, 8, 2), pd.datetime(2011, 2, 1), np.nan])
tenzing_timestamp.summarize(test_series)

In [ ]:
list('wham bam')